In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
# import xgboost as xgb
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import datetime

0it [00:00, ?it/s]


In [22]:
train = pd.read_csv('./train_1.csv').fillna(0)

In [170]:
key = pd.read_csv('./key_trunc.csv')

In [37]:
sub = pd.read_csv("./sample_submission_1.csv")

In [171]:
train_melt = pd.read_csv("train_feats.csv")

In [38]:
for col in train.columns[1:]:
    train[col] = pd.to_numeric(train[col],downcast='integer')

NameError: name 'train' is not defined

In [172]:
def get_median(row, last_n_days):
    return pd.rolling_median(pd.Series(row), window = last_n_days)    

In [173]:
def get_mean(row, last_n_days):
    return pd.rolling_mean(pd.Series(row), window = last_n_days)    

In [174]:
def get_long_stats(func_type, last_n_days, df, df_melt):

    if func_type == "median":
        rolling_stats = df.iloc[:, -60:].progress_apply(get_median, last_n_days = last_n_days, axis = 1)
    elif func_type == "mean":
        rolling_stats = df.iloc[:, -60:].progress_apply(get_mean, last_n_days = last_n_days, axis = 1)

    rolling_stats_df = pd.concat([df.Page, rolling_stats.apply(pd.Series)], axis = 1)    
    rolling_stats_df.columns = ["Page"] + list(df.columns[-60:])
    
    rolling_stats_melt = pd.melt(rolling_stats_df, id_vars=['Page'], var_name="date", value_name=func_type + "_" + str(last_n_days))
    
    df_melt = pd.merge(df_melt, rolling_stats_melt, on = ["Page", "date"])     
    return df_melt

In [175]:
def generate_train_melt():

    train_melt = pd.melt(pd.concat([train.Page, train.iloc[:, -60:]], axis = 1), id_vars=['Page'], var_name="date", value_name="visits")

    train_melt = get_long_stats("mean", 7, train, train_melt)
    train_melt = get_long_stats("mean", 30, train, train_melt)
    train_melt = get_long_stats("mean", 60, train, train_melt)

    train_melt = get_long_stats("median", 7, train, train_melt)
    train_melt = get_long_stats("median", 30, train, train_melt)
    train_melt = get_long_stats("median", 60, train, train_melt)

    train_melt["month"] = train_melt.progress_apply(get_month, axis = 1)
    train_melt["date_num"] = train_melt.progress_apply(get_date, axis = 1)
    train_melt["weekday"] = train_melt.progress_apply(get_weekday, axis = 1)
    # train_melt["language"] = train_melt.progress_apply(get_language, axis = 1)
    
# generate_train_melt()
# train_melt.to_csv("train_melt.csv", index = False)

In [176]:
def get_source(row):
    return row.Page.split["."][-1].split("_")[-1]

In [177]:
def get_language(row):
    try:
        return row.Page.split(".")[0].split("_")[-1]    
    except:
        return "en"

In [178]:
def get_month(row):
    return row.date.split("-")[1]

In [179]:
def get_date(row):
    return row.date.split("-")[2]

In [180]:
def get_weekday(row):
    return datetime.datetime.strptime(row.date, '%Y-%m-%d').date().weekday()

In [28]:
# def get_language_mean(row):
# lang_count = pd.Series(train_melt.groupby(["language"])["Page"].count()).sort_values(axis = 0, ascending = False)
# print lang_count

language
en                                                                                                     1393140
ja                                                                                                     1220940
de                                                                                                     1074780
fr                                                                                                     1058940
zh                                                                                                     1029120
ru                                                                                                      893220
es                                                                                                      838140
www                                                                                                     413280
commons                                                                                                

In [181]:
def get_median_7(row, last_index):
    
    return np.median(row.iloc[last_index - 7 : last_index].tolist())

In [182]:
def get_median_30(row, last_index):
    
    return np.median(row.iloc[last_index - 30 : last_index].tolist())

In [183]:
def get_median_60(row, last_index):
    
    return np.median(row.iloc[last_index - 60 : last_index].tolist())

In [184]:
def get_mean_7(row, last_index):
    
    return np.mean(row.iloc[last_index - 7 : last_index].tolist())

In [185]:
def get_mean_30(row, last_index):
    
    return np.mean(row.iloc[last_index - 30 : last_index].tolist())

In [186]:
def get_mean_60(row, last_index):
    
    return np.mean(row.iloc[last_index - 60 : last_index].tolist())

In [187]:
# def get_test_feats(row, last_index):
    
#     subset_row = row.iloc[last_index - 60 : last_index].tolist()
    
#     median_7 = np.median(subset_row[-7:])
#     median_30 = np.median(subset_row[-30:])
#     median_60 = np.median(subset_row[-60:])

#     mean_7 = np.mean(subset_row[-7:])
#     mean_30 = np.mean(subset_row[-30:])
#     mean_60 = np.mean(subset_row[-60:])
    
#     return pd.Series([row.Page, median_7, median_30, median_60, mean_7, mean_30, mean_60])

In [188]:
def get_preds(test_feats):
    
    return pd.Series([1 for i in range(0, len(test_feats))])

In [191]:
def get_overall_df():
    
#     key = pd.read_csv("key_trunc.csv")
    key_wide = pd.DataFrame()
    key_wide["page"] = np.unique(key.trunc_page).tolist()
    for i in np.unique(key.date).tolist():
        key_wide[i] = 0

    overall_df = pd.merge(train, key_wide, left_on="Page", right_on = "page", how = "inner")
    overall_df.drop("page", axis = 1, inplace = True)
    
    return overall_df

In [196]:
def test_and_pred(overall_df):

    for i in range(overall_df.columns.tolist().index('2017-01-01'), len(overall_df.columns)):

        last_date = overall_df.columns.tolist()[i]
        print last_date

        last_index = overall_df.columns.tolist().index(last_date)

        test_feats = overall_df.Page
        test_feats["median_7"] = overall_df.apply(get_median_7, axis = 1, last_index = last_index)
        test_feats["median_30"] = overall_df.apply(get_median_30, axis = 1, last_index = last_index)
        test_feats["median_60"] = overall_df.apply(get_median_60, axis = 1, last_index = last_index)
        test_feats["mean_7"] = overall_df.apply(get_mean_7, axis = 1, last_index = last_index)
        test_feats["mean_30"] = overall_df.apply(get_mean_30, axis = 1, last_index = last_index)
        test_feats["mean_60"] = overall_df.apply(get_mean_60, axis = 1, last_index = last_index)

        test_feats["month"] = last_date.split("-")[1]
        test_feats["datenum"] = last_date.split("-")[2]
        test_feats["weekday"] = datetime.datetime.strptime(last_date, '%Y-%m-%d').date().weekday()

        prediction = get_preds(test_feats)

        overall_df[last_date] = prediction
        #preds = pd.concat([preds, prediction])
        
    return overall_df

In [195]:
overall_df = get_overall_df()
overall_df = test_and_pred(overall_df)

0         1
1         1
2         1
3         1
4         1
5         1
6         1
7         1
8         1
9         1
10        1
11        1
12        1
13        1
14        1
15        1
16        1
17        1
18        1
19        1
20        1
21        1
22        1
23        1
24        1
25        1
26        1
27        1
28        1
29        1
         ..
145033    1
145034    1
145035    1
145036    1
145037    1
145038    1
145039    1
145040    1
145041    1
145042    1
145043    1
145044    1
145045    1
145046    1
145047    1
145048    1
145049    1
145050    1
145051    1
145052    1
145053    1
145054    1
145055    1
145056    1
145057    1
145058    1
145059    1
145060    1
145061    1
145062    1
Name: 2017-01-01, Length: 145063, dtype: int64

In [ ]:
# train_melt = pd.read_csv("train_feats.csv")
key = pd.read_csv("key_trunc.csv")

train_melt = pd.merge(train_melt, key, left_on="Page", right_on="trunc_page", how="inner")
train_melt.drop("Page_x", inplace=True, axis = 1)
train_melt.drop("Page_y", inplace=True, axis = 1)

train_melt.to_csv("train_merged.csv", index = False)

In [93]:
train_merged_filtered = train_merged[["Id", "median_60_days"]]

In [111]:
sub = pd.merge(sub, train_merged_filtered, on = "Id")
sub.drop("Visits", inplace=True, axis = 1)
sub.columns = ["Id", "Visits"]

In [112]:
sub.to_csv("median_60_days.csv", index = False)

In [ ]:
def run_xgb(x_train, x_test, x_label):

    # x_train = pd.concat([pos_train, neg_train]) #Concat positive and negative
    # y_train = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist() #Putting in 1 and 0

    # x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=4242)

    # Set our parameters for xgboost
    params = {}
    params['objective'] = 'binary:logistic'
    params['eval_metric'] = 'logloss'
    params['eta'] = 0.05
    params['max_depth'] = 6
    params['silent'] = 1

    d_train = xgb.DMatrix(x_train, label=x_label)
    d_test = xgb.DMatrix(x_test)

    watchlist = [(d_train, 'train')]

    bst = xgb.train(params, d_train, 100, watchlist, early_stopping_rounds=50, verbose_eval=50)

    p_test = bst.predict(d_test)

    # xgb.plot_importance(bst)
    # pyplot.show()

    return p_test